In [1]:
from warnings import filterwarnings

filterwarnings('ignore')

In [2]:
from copy import deepcopy
import pandas as pd
import numpy as np
from more_itertools import flatten

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [5]:
trade_df = pd.read_csv('data/dyadic_trade_3.0.csv', encoding = 'utf8')

trade_df.rename({'flow1': 'money_flow_1', 'flow2': 'money_flow_2',
         'ccode1': 'c_code_1', 'ccode2': 'c_code_2',
         'importer1': 'state_name_1', 'importer2': 'state_name_2'},
        axis = 1, inplace = True)

trade_df['c_code_1'] = trade_df['c_code_1'].astype(int)
trade_df['c_code_2'] = trade_df['c_code_2'].astype(int)

trade_df['money_flow_1'] = ([s * 1000000 for s in trade_df['money_flow_1']])
trade_df['money_flow_2'] = ([s * 1000000 for s in trade_df['money_flow_2']])

trade_df.loc[trade_df['money_flow_1'] == -9000000, 'money_flow_1'] = None
trade_df.loc[trade_df['money_flow_2'] == -9000000, 'money_flow_2'] = None

trade_df = trade_df.sort_values(by = 'year', ascending = True).reset_index()

trade_df.drop(['source1', 'source2', 'bel_lux_alt_flow1', 'bel_lux_alt_flow2', 'china_alt_flow1', 'china_alt_flow2', 'version', 'index'], axis = 1, inplace = True)

In [6]:
trade_df_copy = deepcopy(trade_df)

trade_df_copy.rename({'c_code_1': 'c_code_1_copy', 'c_code_2':'c_code_2_copy',
         'state_name_1': 'state_name_1_copy', 'state_name_2': 'state_name_2_copy',
         'money_flow_1': 'money_flow_1_copy', 'money_flow_2': 'money_flow_2_copy'},
        axis = 1, inplace = True)

trade_df_copy.rename({'c_code_1_copy': 'c_code_2', 'c_code_2_copy':'c_code_1',
         'state_name_1_copy': 'state_name_2', 'state_name_2_copy': 'state_name_1',
         'money_flow_1_copy': 'money_flow_2', 'money_flow_2_copy': 'money_flow_1'},
        axis = 1, inplace = True)

In [7]:
trade_df = pd.concat([trade_df, trade_df_copy], axis = 0)

trade_df = trade_df.sort_values(by = 'year', ascending = True).reset_index()

trade_df.drop(['index'], axis = 1, inplace = True)

In [8]:
trade_df.head()

,c_code_1,c_code_2,money_flow_1,money_flow_2,state_name_1,state_name_2,year
0,210,390,nan,nan,Netherlands,Denmark,1870
1,255,160,190000.00,nan,Germany,Argentina,1870
2,200,70,1450000.00,5110000.00,United Kingdom,Mexico,1870
3,600,255,nan,nan,Morocco,Germany,1870
4,350,135,nan,nan,Greece,Peru,1870


In [9]:
aggregations = {
  'c_code_2': 'count',
  'money_flow_1': 'sum',
  'money_flow_2': 'sum'
  }

trade_df_group = trade_df.groupby(['c_code_1', 'state_name_1', 'year']).agg(aggregations).reset_index()

trade_df_group.rename({'c_code_2': 'num_trade_states', 'money_flow_1': 'export_dollars', 'money_flow_2': 'import_dollars'}, axis = 1, inplace = True)

In [10]:
trade_df_group.to_pickle('pickle/trade_df_group.pkl')

In [11]:
trade_df_group.head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars
0,2,United States of America,1870,36,256170000.00,359400000.00
1,2,United States of America,1871,35,319960000.00,430510000.00
2,2,United States of America,1872,32,313530000.00,353040000.00
3,2,United States of America,1873,32,392890000.00,542810000.00
4,2,United States of America,1874,32,317460000.00,553420000.00


In [12]:
mat_cap_df = pd.read_csv('data/NMC_5_0-wsupplementary.csv', encoding = 'latin-1')

mat_cap_df.rename({'milex': 'military_expenditure', 'milper': 'military_personnel',
          'irst': 'iron_steel_prod', 'pec': 'prim_energy_consumption',
          'tpop': 'total_pop', 'upop': 'urban_pop',
          'upopgrowth': 'urban_pop_growth_rate', 'ccode': 'c_code_1',
          'statenme': 'state_name_1', 'cinc': 'cinc_score'}, axis = 1, inplace = True)

mat_cap_df['military_expenditure'] = ([s * 1000 for s in mat_cap_df['military_expenditure']])
mat_cap_df['military_personnel'] = ([s * 1000 for s in mat_cap_df['military_personnel']])
mat_cap_df['total_pop'] = ([s * 1000 for s in mat_cap_df['total_pop']])
mat_cap_df['urban_pop'] = ([s * 1000 for s in mat_cap_df['urban_pop']])
mat_cap_df['iron_steel_prod'] = ([s * 2000000 for s in mat_cap_df['iron_steel_prod']])
mat_cap_df['prim_energy_consumption'] = ([s * 2000000 for s in mat_cap_df['prim_energy_consumption']])

mat_cap_df.loc[mat_cap_df['military_expenditure'] == -9000, 'military_expenditure'] = None
mat_cap_df.loc[mat_cap_df['military_personnel'] == -9000, 'military_personnel'] = None
mat_cap_df.loc[mat_cap_df['total_pop'] == -9000, 'total_pop'] = None
mat_cap_df.loc[mat_cap_df['urban_pop'] == -9000, 'urban_pop'] = None
mat_cap_df.loc[mat_cap_df['iron_steel_prod'] == -18000000 , 'iron_steel_prod'] = None
mat_cap_df.loc[mat_cap_df['prim_energy_consumption'] == -18000000 , 'prim_energy_consumption'] = None

mat_cap_df = mat_cap_df.sort_values(by = 'year', ascending = True).reset_index()

mat_cap_df.drop(['stateabb', 'milpersource', 'milpernote', 'milexsource', 'milexnote',
         'irstsource', 'irstnote', 'pecsource', 'pecnote', 'pecnote',
         'tpopsource', 'tpopnote', 'upopsource', 'upopnote', 'upopgrowthsource',
         'irstqualitycode', 'irstanomalycode', 'pecqualitycode', 'pecanomalycode',
         'tpopqualitycode', 'tpopanomalycode', 'upopqualitycode', 'upopanomalycode',
         'version', 'urban_pop_growth_rate', 'index'], axis = 1, inplace = True)

In [13]:
mat_cap_df.to_pickle('pickle/mat_cap_df.pkl')

In [14]:
mat_cap_df.head()

,state_name_1,c_code_1,year,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score
0,United States of America,2,1816,3823000.00,17000.00,160000000.00,508000000.00,8659000.00,101000.00,0.04
1,Spain,230,1816,6512000.00,125000.00,20000000.00,0.00,11073000.00,221000.00,0.05
2,Netherlands,210,1816,2375000.00,26000.00,100000000.00,2284000000.00,5610000.00,337000.00,0.04
3,Portugal,235,1816,nan,22000.00,0.00,0.00,2746000.00,179000.00,0.01
4,United Kingdom,200,1816,16942000.00,255000.00,540000000.00,45056000000.00,19520000.00,1957000.00,0.34


In [15]:
alliance_df = pd.read_csv('data/alliance_v4.1_by_directed_yearly.csv', encoding = 'utf8')

alliance_df.rename({'left_censor': 'pre_1816_alliance', 'right_censor': 'in_effect_1231_2012',
          'neutrality': 'neutrality_treaty', 'nonaggression': 'nonaggression_treaty',
          'defense': 'defense_treaty', 'entente': 'entente_treaty',
          'ccode1': 'c_code_1', 'ccode2': 'c_code_2', 'state_name1': 'state_name_1',
          'state_name2': 'state_name_2'}, axis = 1, inplace = True)

alliance_df['alliance_date_start'] = pd.to_datetime((alliance_df['dyad_st_year']*10000+alliance_df['dyad_st_month']*100+alliance_df['dyad_st_day']).apply(str),format='%Y%m%d')
alliance_df['alliance_date_end'] = pd.to_datetime((alliance_df['dyad_end_year']*10000+alliance_df['dyad_end_month']*100+alliance_df['dyad_end_day']).apply(str),format='%Y%m%d')

alliance_df = alliance_df.sort_values(by = 'year', ascending = True).reset_index()

alliance_df.drop(['dyad_st_day', 'dyad_st_month', 'dyad_st_year', 'dyad_end_day',
         'dyad_end_month', 'dyad_end_year', 'version4id', 'version', 'index'],
         axis = 1, inplace = True)

In [16]:
alliance_df.head()

,c_code_1,state_name_1,c_code_2,state_name_2,pre_1816_alliance,in_effect_1231_2012,defense_treaty,neutrality_treaty,nonaggression_treaty,entente_treaty,year,alliance_date_start,alliance_date_end
0,200,United Kingdom,235,Portugal,1,1,1,0,1.00,0.00,1816,1816-01-01,NaT
1,275,Hesse Grand Ducal,300,Austria-Hungary,1,0,1,0,1.00,1.00,1816,1816-01-01,1848-03-15
2,275,Hesse Grand Ducal,273,Hesse Electoral,1,0,1,0,1.00,1.00,1816,1816-01-01,1848-03-15
3,275,Hesse Grand Ducal,271,Wuerttemburg,1,0,1,0,1.00,1.00,1816,1816-01-01,1848-03-15
4,275,Hesse Grand Ducal,269,Saxony,1,0,1,0,1.00,1.00,1816,1816-01-01,1848-03-15


In [17]:
aggregations = {
  'c_code_2': 'count',
  'pre_1816_alliance': 'sum',
  'in_effect_1231_2012': 'sum',
  'defense_treaty': 'sum',
  'neutrality_treaty': 'sum',
  'nonaggression_treaty': 'sum',
  'entente_treaty': 'sum'
  }

alliance_df_group = alliance_df.groupby(['c_code_1', 'state_name_1', 'year']).agg(aggregations).reset_index()

alliance_df_group = alliance_df_group.sort_values(by = 'year', ascending = True).reset_index()

alliance_df_group.rename({'c_code_2': 'num_alliances', 'pre_1816_alliance': 'pre_1816_alliances',
             'in_effect_1231_2012': 'num_in_effect_1231_2012',
             'defense_treaty': 'defense_treaties',
             'neutrality_treaty': 'neutrality_treaties',
             'nonaggression_treaty': 'nonaggression_treaties',
             'entente_treaty': 'entente_treaties'}, axis = 1, inplace = True)

alliance_df_group.drop(['index'], axis = 1, inplace = True)

In [18]:
alliance_df_group.to_pickle('pickle/alliance_df_group.pkl')

In [19]:
alliance_df_group.head()

,c_code_1,state_name_1,year,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties
0,230,Spain,1816,1,0,0,1,0,0.00,0.00
1,255,Germany,1816,10,10,0,10,0,7.00,10.00
2,267,Baden,1816,7,7,0,7,0,7.00,7.00
3,271,Wuerttemburg,1816,7,7,0,7,0,7.00,7.00
4,380,Sweden,1816,1,1,0,0,0,0.00,1.00


In [20]:
contiguity_df = pd.read_csv('data/contdird.csv', encoding = 'utf8')

contiguity_df.rename({'state1no': 'c_code_1', 'state2no': 'c_code_2',
           'state1ab': 'state_name_abb_1', 'state2ab': 'state_name_abb_2',
           'conttype': 'contiguity_type'}, axis = 1, inplace = True)
         
contiguity_df.drop(['dyad', 'version'], axis = 1, inplace = True)

In [21]:
contiguity_df.to_pickle('pickle/contiguity_df.pkl')

In [22]:
contiguity_df.head()

,c_code_1,state_name_abb_1,c_code_2,state_name_abb_2,year,contiguity_type
0,2,USA,20,CAN,1920,1
1,2,USA,20,CAN,1921,1
2,2,USA,20,CAN,1922,1
3,2,USA,20,CAN,1923,1
4,2,USA,20,CAN,1924,1


In [23]:
mid_df = pd.read_csv('data/dyadic MIDs 3.1.csv', encoding = 'utf8')
  
mid_df.rename({'revstata': 'revision_sought', 'revtypea': 'revision_type',
        'fatleva': 'state_fatality_bin', 'highmcaa': 'highest_mca',
        'hihosta': 'highest_hostility', 'durindx': 'year_num',
        'midc3hia': 'highest_action_for_state', 'orignata': 'origin_participant',
        'notarg': 'num_against', 'settlmnt': 'settlement_type',
        'fatlev': 'total_fatality_bin', 'noinit': 'num_with',
        'disno': 'dispute_id', 'statea': 'c_code_1', 'stateb': 'c_code_2',
        'namea': 'state_name_abb_a', 'nameb': 'state_name_abb_b',
        'cumdurat': 'cumulative_duration', 'ongo2010': 'ongoing_2010'}, axis = 1, inplace = True)

mid_df['date_start'] = pd.to_datetime((mid_df['strtyr']*10000+mid_df['strtmnth']*100+mid_df['strtday']).apply(str),format='%Y%m%d')
mid_df['date_end'] = pd.to_datetime((mid_df['endyear']*10000+mid_df['endmnth']*100+mid_df['endday']).apply(str),format='%Y%m%d')

mid_df.loc[mid_df['sideaa'] == 1, 'initiator'] = 1
mid_df.loc[mid_df['sideaa'] != 1, 'initiator'] = 0

mid_df = mid_df.sort_values(by = 'year', ascending = True).reset_index()

mid_df.drop(['strtmnth', 'strtday', 'strtyr', 'endmnth', 'endday',
       'endyear', 'dyindex', 'duration', 'disno4', 'sideaa',
       'sideab', 'revstatb', 'revtypeb', 'fatlevb', 'highmcab',
       'hihostb', 'hihost', 'orignatb', 'recip', 'rolea', 'roleb',
       'mid3hiact', 'mid3hib', 'change', 'changetype_1',
       'changetype_2', 'highact', 'new', 'mid3hia', 'index'],
      axis = 1, inplace = True)

In [24]:
mid_df.head()

,dispute_id,c_code_1,state_name_abb_a,c_code_2,state_name_abb_b,year,outcome,settlement_type,total_fatality_bin,num_with,num_against,revision_sought,revision_type,state_fatality_bin,highest_mca,highest_hostility,origin_participant,war,year_num,cumulative_duration,ongoing_2010,date_start,date_end,initiator
0,3239,230,SPN,2,USA,1816,2,3,0,1,1,0,0,0,1,1,1,0,1,1,0,1816-07-27,1816-07-27,0.00
1,3239,2,USA,230,SPN,1816,1,3,0,1,1,1,2,0,19,4,1,0,1,1,0,1816-07-27,1816-07-27,1.00
2,3321,365,RUS,640,TUR,1817,4,1,0,1,1,1,1,0,7,3,1,0,1,51,0,1817-09-12,1817-11-01,1.00
3,3321,640,TUR,365,RUS,1817,3,1,0,1,1,0,0,0,13,3,1,0,1,51,0,1817-09-12,1817-11-01,0.00
4,1567,230,SPN,2,USA,1818,2,1,-9,1,1,0,0,-9,19,4,1,0,1,77,0,1818-03-15,1818-05-30,0.00


In [25]:
aggregations = {
  'c_code_2': 'count',
  'num_with': 'sum',
  'num_against': 'sum',
  'cumulative_duration': 'sum',
  'war': 'sum',
  'origin_participant': 'mean',
  'ongoing_2010': 'mean',
  'revision_sought': 'mean',
  'revision_type': pd.Series.mode,
  'state_fatality_bin': pd.Series.mode,
  'outcome': pd.Series.mode,
  'settlement_type': pd.Series.mode,
  'highest_mca': pd.Series.mode,
  'highest_hostility': pd.Series.mode
  }

mid_df_group = mid_df.groupby(['c_code_1', 'state_name_abb_a', 'year']).agg(aggregations).reset_index()

for i, country in enumerate(mid_df_group['c_code_1']):
    mid_df_group.loc[i, 'num_with'] = mid_df_group['num_with'][i]/mid_df_group['c_code_2'][i]
    mid_df_group.loc[i, 'num_against'] = mid_df_group['num_against'][i]/mid_df_group['c_code_2'][i]
    mid_df_group.loc[i, 'cumulative_duration'] = mid_df_group['cumulative_duration'][i]/mid_df_group['c_code_2'][i]
    mid_df_group.loc[i, 'cumulative_duration'] = mid_df_group['cumulative_duration'][i]/365

mid_df_group = mid_df_group.sort_values(by = 'year', ascending = True).reset_index()

mid_df_group.rename({'c_code_2': 'num_conflicts', 'num_with': 'avg_with', 'num_against': 'avg_against',
           'cumulative_duration': 'avg_cum_duration', 'war': 'num_wars',
           'origin_participant': 'origin_participant_pct', 'revision_sought': 'revision_pct',
          'revision_type': 'revision_type_mode', 'state_fatality_bin': 'state_fatality_bin_mode',
           'outcome': 'outcome_mode', 'settlement_type': 'settlement_type_mode',
           'highest_mca': 'highest_mca_mode', 'highest_hostility': 'highest_hostility_mode'}, axis = 1, inplace = True)

mid_df_group.drop(['index'], axis = 1, inplace = True)

In [26]:
mid_df_group.to_pickle('pickle/mid_df_group.pkl')

In [27]:
mid_df_group.head()

,c_code_1,state_name_abb_a,year,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode
0,2,USA,1816,1,1.00,1.00,0.00,0,1.00,0.00,1.00,2,0,1,3,19,4
1,230,SPN,1816,1,1.00,1.00,0.00,0,1.00,0.00,0.00,0,0,2,3,1,1
2,365,RUS,1817,1,1.00,1.00,0.14,0,1.00,0.00,1.00,1,0,4,1,7,3
3,640,TUR,1817,1,1.00,1.00,0.14,0,1.00,0.00,0.00,0,0,3,1,13,3
4,2,USA,1818,1,1.00,1.00,0.21,0,1.00,0.00,1.00,1,-9,1,1,16,4


In [28]:
gov_df = pd.read_csv('data/ddrevisited_data_v1.csv', encoding = 'latin-1')

gov_df.rename({'chgterr': 'territory_change', 'ychgterr': 'territory_change_year',
        'entryy': 'first_recorded_year', 'exity': 'last_recorded_year',
        'bornyear': 'born_year', 'endyear': 'died_year',
        'exselec': 'election_type', 'legselec': 'legislation_type',
        'closed': 'legislature_status', 'dejure': 'party_legal_status',
        'defacto': 'party_existance', 'defacto2': 'party_existance_outside_regime', 
        'lparty': 'legislature_parties', 'incumb': 'incumbent_type',
        'collect': 'collective_leadership', 'eheads': 'num_leadership_changes',
        'ehead': 'leader_name', 'epost': 'post_name', 'edate': 'entrance_date',
        'ageeh': 'leader_tenure', 'emil': 'military_leader',
        'royal': 'royal_leader', 'comm': 'communist_leader',
        'edeath': 'leader_died', 'democracy': 'democratic_regime',
        'assconfid': 'cabinet_assembly', 'poppreselec': 'popular_election',
        'regime': 'regime_type', 'ttd': 'transition_democracy',
        'tta': 'transition_dictatorship', 'agedem': 'age_govt',
        'stra': 'num_transitions_ever', 'cowcode': 'c_code_1',
        'ctryname': 'state_name_1', 'headdiff': 'nominal_vs_eff_diff',
        'cowcode2': 'c_code_2'},
       axis = 1, inplace = True)

gov_df.drop(['order', 'aclpcode', 'c_code_2', 'qogctycode', 'qogctylett',
       'qogctyyear', 'ccdcodelet', 'ccdcodenum', 'aclpyear',
       'cowcode2year', 'cowcodeyear', 'flagc_cowcode2', 'flage_cowcode2',
       'imf_code', 'politycode', 'bankscode', 'dpicode', 'uncode', 'un_region',
       'un_region_name', 'un_continent', 'last_recorded_year',
       'dupcow', 'dupwdi', 'dupun', 'dupdpi', 'dupimf', 'dupbanks',
       'cid', 'wdicode', 'un_continent_name', 'aclp_region', 'type2',
       'nheads', 'nmil', 'nhead', 'npost', 'ndate', 'entrance_date',
       'tenure08', 'ecens08', 'flageh', 'tt', 'flagc', 'flagdem',
       'flagreg', 'agereg', 'first_recorded_year', 'born_year',
       'died_year', 'leader_name', 'post_name'], axis = 1, inplace = True)

In [29]:
gov_df.to_pickle('pickle/gov_df.pkl')

In [30]:
gov_df.head()

,state_name_1,year,c_code_1,territory_change,territory_change_year,election_type,legislation_type,legislature_status,party_legal_status,party_existance,party_existance_outside_regime,legislature_parties,incumbent_type,collective_leadership,num_leadership_changes,leader_tenure,military_leader,royal_leader,nominal_vs_eff_diff,communist_leader,leader_died,democratic_regime,cabinet_assembly,popular_election,regime_type,transition_democracy,transition_dictatorship,age_govt,num_transitions_ever
0,Afghanistan,1946.00,700.00,0.00,0.00,3.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,14.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,5.00,0.00,0.00,18.00,0.00
1,Afghanistan,1947.00,700.00,0.00,0.00,3.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,15.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,5.00,0.00,0.00,19.00,0.00
2,Afghanistan,1948.00,700.00,0.00,0.00,3.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,5.00,0.00,0.00,20.00,0.00
3,Afghanistan,1949.00,700.00,0.00,0.00,3.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,17.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,5.00,0.00,0.00,21.00,0.00
4,Afghanistan,1950.00,700.00,0.00,0.00,3.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,5.00,0.00,0.00,22.00,0.00


In [31]:
wrp_df = pd.read_csv('data/WRP_national.csv', encoding = 'utf8')

wrp_df.rename({'judgen': 'total_jewish', 'chrstprot': 'total_protestant',
        'chrstcat': 'total_catholic', 'chrstgen': 'total_christian',
        'pop': 'total_population', 'nonrelig': 'total_non_religious',
        'budgen': 'total_buddhist', 'hindgen': 'total_hindu',
        'islmgen': 'total_islam', 'islmsun': 'total_sunni',
        'islmshi': 'total_shia', 'name': 'state_name_1',
        'state': 'c_code_1', 'shntgen': 'total_shinto',
        'sikhgen': 'total_sikh'}, axis = 1, inplace = True)

wrp_df.drop(['jdcons', 'judconspct', 'judgenpct', 'judorth',
      'judorthpct', 'judothr', 'judothrpct', 'judref',
      'judrefpct', 'chrstang', 'chrstangpct',
       'chrstcatpct', 'chrstgenpct', 'chrstorth',
       'chrstorthpct', 'chrstothr', 'chrstothrpct',
       'chrstprotpct', 'confgen', 'confgenpct',
       'Version', 'zorogen', 'zorogenpct', 'anmgen',
       'anmgenpct', 'bahgen', 'bahgenpct', 'budgenpct',
       'budmah', 'budmahpct', 'budothr', 'budothrpct',
       'budthr', 'budthrpct', 'datatype', 'dualrelig',
       'hindgenpct', 'islmahm', 'islmahmpct', 'islmalw',
       'islmalwpct', 'islmgenpct', 'islmibd', 'islmibdpct',
       'islmnat', 'islmnatpct', 'islmothr', 'islmothrpct',
       'islmshipct', 'islmsunpct', 'jaingen', 'jaingenpct',
       'nonreligpct', 'othrgen', 'othrgenpct', 'recreliab',
       'reliabilevel', 'shntgenpct', 'sikhgenpct',
       'sourcecode', 'sourcereliab', 'sumrelig', 'sumreligpct',
       'syncgen', 'syncgenpct', 'taogen', 'taogenpct',
       'total', 'total_population'], axis = 1, inplace = True)

In [32]:
wrp_df.head()

,year,c_code_1,state_name_1,total_protestant,total_catholic,total_christian,total_jewish,total_sunni,total_shia,total_islam,total_buddhist,total_hindu,total_sikh,total_shinto,total_non_religious
0,1945,2,USA,66069671,38716742,110265118,4641182,0,0,0,1601218,0,0,0,22874544
1,1950,2,USA,73090083,42635882,122994019,6090837,0,0,0,0,0,0,0,22568130
2,1955,2,USA,79294628,46402368,134001770,5333332,0,0,0,90173,0,0,0,23303540
3,1960,2,USA,90692928,50587880,150234347,5500000,0,0,0,2012131,0,0,0,21548225
4,1965,2,USA,94165803,64761783,167515758,5600000,0,0,0,1080892,0,0,0,19852362


In [33]:
c_code_df = pd.read_csv('data/COW country codes.csv', encoding = 'utf8')

c_code_df.rename({'CCode': 'c_code', 'StateNme': 'state_name'}, axis = 1, inplace = True)

c_code_df.drop(['StateAbb'], axis = 1, inplace = True)

In [34]:
c_code_dic = {}

for i, code in enumerate(c_code_df['c_code']):
    c_code_dic[code] = c_code_df['state_name'][i]

In [35]:
row_list = []
years = list(np.linspace(1945, 2010, 14))
for c_code in wrp_df['c_code_1'].unique():
    for year in years:
        try:
            assert len(list(flatten(wrp_df.loc[(wrp_df['year'] == year) & (wrp_df['c_code_1'] == c_code)].values))) > 0
            row_list.append(list(flatten(wrp_df.loc[(wrp_df['year'] == year) & (wrp_df['c_code_1'] == c_code)].values)))
        except:
            row_list.append([year, c_code, c_code_dic[c_code], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [36]:
wrp_df = pd.DataFrame(row_list)

wrp_df.columns = ['year', 'c_code_1', 'state_name_1', 'total_protestant',
         'total_catholic', 'total_christian', 'total_jewish',
         'total_sunni', 'total_shia', 'total_islam',
         'total_buddhist', 'total_hindu', 'total_sikh',
         'total_shinto', 'total_non_religious']

In [37]:
years = list(np.linspace(1945, 2010, 14))

column_list = ['total_protestant', 'total_catholic', 'total_christian',
        'total_jewish', 'total_sunni', 'total_shia',
        'total_islam', 'total_buddhist', 'total_hindu',
        'total_sikh','total_shinto', 'total_non_religious']

In [38]:
fix_count = 0

for c_code in wrp_df['c_code_1'].unique():
    for column in column_list:
        data_list = []
        for year in years:
            data_list.append(wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == year)][column].values[0])
        for i, item in enumerate(data_list[1:]):
            i+=1
            if item == 0 and data_list.count(0) == 13:
                pass
            elif item == 0 and data_list[i-1] != 0:
                if i+1 <= 13 and data_list[i+1] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+1])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])/2)
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                elif i+2 <= 13 and data_list[i+2] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+2])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])/3)
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+1]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                elif i+3 <= 13 and data_list[i+3] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+3])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])/4)
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+1]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+2]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                elif i+4 <= 13 and data_list[i+4] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+4])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])/5)
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+1]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+2]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+3]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                elif i+5 <= 13 and data_list[i+5] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+5])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])/6)
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+1]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+2]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+3]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+4]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                elif i+6 <= 13 and data_list[i+6] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+6])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])/7)
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+1]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+2]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+3]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+4]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+5]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                elif i+7 <= 13 and data_list[i+7] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+7])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])/8)
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+1]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+2]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+3]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+4]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+5]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+6]), column] = (7 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                elif i+8 <= 13 and data_list[i+8] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+8])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])/9)
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+1]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+2]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+3]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+4]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+5]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+6]), column] = (7 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+7]), column] = (8 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                elif i+9 <= 13 and data_list[i+9] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+9])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])/10)
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+1]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+2]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+3]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+4]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+5]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+6]), column] = (7 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+7]), column] = (8 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+8]), column] = (9 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                elif i+10 <= 13 and data_list[i+10] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+10])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])/11)
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+1]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+2]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+3]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+4]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+5]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+6]), column] = (7 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+7]), column] = (8 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+8]), column] = (9 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+9]), column] = (10 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                elif i+11 <= 13 and data_list[i+11] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+11])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])/12)
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+1]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+2]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+3]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+4]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+5]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+6]), column] = (7 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+7]), column] = (8 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+8]), column] = (9 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+9]), column] = (10 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+10]), column] = (11 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                elif i+12 <= 13 and data_list[i+12] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+12])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])/13)
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+1]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+2]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+3]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+4]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+5]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+6]), column] = (7 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+7]), column] = (8 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+8]), column] = (9 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+9]), column] = (10 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+10]), column] = (11 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i+11]), column] = (12 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[i-1])][column].values[0])
                else:
#                     print(c_code, years[i], column)
                    fix_count+=1
                    
print(fix_count)

40 2000.0 total_buddhist
42 2010.0 total_islam
42 2010.0 total_buddhist
52 2010.0 total_jewish
52 2010.0 total_buddhist
54 2005.0 total_islam
54 2010.0 total_buddhist
54 2010.0 total_hindu
58 2010.0 total_islam
58 2010.0 total_hindu
70 2010.0 total_buddhist
70 2010.0 total_hindu
80 2000.0 total_jewish
90 2010.0 total_buddhist
91 2010.0 total_buddhist
92 2005.0 total_buddhist
93 2005.0 total_jewish
93 2005.0 total_buddhist
95 1995.0 total_non_religious
110 2010.0 total_buddhist
115 2010.0 total_buddhist
115 2010.0 total_non_religious
205 2010.0 total_hindu
260 1995.0 total_protestant
260 1995.0 total_catholic
260 1995.0 total_christian
260 1995.0 total_jewish
260 1995.0 total_islam
260 1995.0 total_non_religious
265 1990.0 total_protestant
265 1990.0 total_catholic
265 1990.0 total_christian
265 1990.0 total_jewish
265 1990.0 total_non_religious
305 2000.0 total_buddhist
310 2010.0 total_buddhist
315 1995.0 total_protestant
315 1995.0 total_catholic
315 1995.0 total_christian
315 1990.0

In [39]:
fix_count = 0

for c_code in wrp_df['c_code_1'].unique():
    for column in column_list:
        data_list = []
        for year in years:
            data_list.append(wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == year)][column].values[0])
        for i, item in enumerate(data_list[1:]):
            i+=1
            if item == 0 and data_list.count(0) == 13:
                pass
            elif item == 0 and data_list[i-1] != 0:
                if data_list[-1] == 0 and data_list[-2] != 0 and data_list[-3] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-2])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3])][column].values[0]))
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-1]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-2])][column].values[0])
                elif data_list[-1] == 0 and data_list[-3] != 0 and data_list[-4] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4])][column].values[0]))
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-2]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-1]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3])][column].values[0])
                elif data_list[-1] == 0 and data_list[-4] != 0 and data_list[-5] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5])][column].values[0]))
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-2]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-1]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4])][column].values[0])
                elif data_list[-1] == 0 and data_list[-5] != 0 and data_list[-6] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6])][column].values[0]))
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-2]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-1]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5])][column].values[0])
                elif data_list[-1] == 0 and data_list[-6] != 0 and data_list[-7] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7])][column].values[0]))
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-2]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-1]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6])][column].values[0])
                elif data_list[-1] == 0 and data_list[-7] != 0 and data_list[-8] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8])][column].values[0]))
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-2]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-1]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7])][column].values[0])
                elif data_list[-1] == 0 and data_list[-8] != 0 and data_list[-9] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9])][column].values[0]))
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-2]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-1]), column] = (7 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8])][column].values[0])
                elif data_list[-1] == 0 and data_list[-9] != 0 and data_list[-10] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10])][column].values[0]))
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-2]), column] = (7 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-1]), column] = (8 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9])][column].values[0])
                elif data_list[-1] == 0 and data_list[-10] != 0 and data_list[-11] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11])][column].values[0]))
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3]), column] = (7 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-2]), column] = (8 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-1]), column] = (9 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10])][column].values[0])
                elif data_list[-1] == 0 and data_list[-11] != 0 and data_list[-12] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0]))
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4]), column] = (7 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3]), column] = (8 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-2]), column] = (9 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-1]), column] = (10 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11])][column].values[0])
                elif data_list[-1] == 0 and data_list[-12] != 0 and data_list[-13] != 0:
                    rate_of_change = ((wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0] - wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-13])][column].values[0]))
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-11]), column] = rate_of_change + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-10]), column] = (2 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-9]), column] = (3 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-8]), column] = (4 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-7]), column] = (5 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-6]), column] = (6 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-5]), column] = (7 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-4]), column] = (8 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-3]), column] = (9 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-2]), column] = (10 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0])
                    wrp_df.loc[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-1]), column] = (11 * rate_of_change) + (wrp_df[(wrp_df['c_code_1'] == c_code) & (wrp_df['year'] == years[-12])][column].values[0])
                else:
                    fix_count+=1

print(fix_count)

0


In [40]:
wrp_df[wrp_df['total_non_religious'] == 0].head()

,year,c_code_1,state_name_1,total_protestant,total_catholic,total_christian,total_jewish,total_sunni,total_shia,total_islam,total_buddhist,total_hindu,total_sikh,total_shinto,total_non_religious
14,1945.00,20,CAN,4002534.00,5038280.00,10531961.00,176690.00,0.00,0.00,0.00,115066.00,0.00,5000.00,0.00,0.00
28,1945.00,31,Bahamas,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
29,1950.00,31,Bahamas,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
30,1955.00,31,Bahamas,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
31,1960.00,31,Bahamas,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [41]:
row_list = []
years = list(np.linspace(1945, 2010, 66).astype(int))
for c_code in wrp_df['c_code_1'].unique():
    for year in years:
        try:
            assert len(list(flatten(wrp_df.loc[(wrp_df['year'] == year) & (wrp_df['c_code_1'] == c_code)].values))) > 0
            row_list.append(list(flatten(wrp_df.loc[(wrp_df['year'] == year) & (wrp_df['c_code_1'] == c_code)].values)))
        except:
            slope_coef = int(str(year)[-1])
            if slope_coef > 5:
                slope_coef_adj = slope_coef - 5
            else:
                slope_coef_adj = slope_coef
            total_protestant_est = float((slope_coef_adj * ((wrp_df.loc[(wrp_df['year'] == year + (5 - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_protestant'].values - wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_protestant'].values)/5)) + wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_protestant'].values)
            total_catholic_est = float((slope_coef_adj * ((wrp_df.loc[(wrp_df['year'] == year + (5 - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_catholic'].values - wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_catholic'].values)/5)) + wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_catholic'].values)
            total_christian_est = float((slope_coef_adj * ((wrp_df.loc[(wrp_df['year'] == year + (5 - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_christian'].values - wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_christian'].values)/5)) + wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_christian'].values)
            total_jewish_est = float((slope_coef_adj * ((wrp_df.loc[(wrp_df['year'] == year + (5 - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_jewish'].values - wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_jewish'].values)/5)) + wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_jewish'].values)
            total_sunni_est = float((slope_coef_adj * ((wrp_df.loc[(wrp_df['year'] == year + (5 - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_sunni'].values - wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_sunni'].values)/5)) + wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_sunni'].values)
            total_shia_est = float((slope_coef_adj * ((wrp_df.loc[(wrp_df['year'] == year + (5 - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_shia'].values - wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_shia'].values)/5)) + wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_shia'].values)
            total_islam_est = float((slope_coef_adj * ((wrp_df.loc[(wrp_df['year'] == year + (5 - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_islam'].values - wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_islam'].values)/5)) + wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_islam'].values)
            total_buddhist_est = float((slope_coef_adj * ((wrp_df.loc[(wrp_df['year'] == year + (5 - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_buddhist'].values - wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_buddhist'].values)/5)) + wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_buddhist'].values)
            total_hindu_est = float((slope_coef_adj * ((wrp_df.loc[(wrp_df['year'] == year + (5 - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_hindu'].values - wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_hindu'].values)/5)) + wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_hindu'].values)
            total_sikh_est = float((slope_coef_adj * ((wrp_df.loc[(wrp_df['year'] == year + (5 - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_sikh'].values - wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_sikh'].values)/5)) + wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_sikh'].values)
            total_shinto_est = float((slope_coef_adj * ((wrp_df.loc[(wrp_df['year'] == year + (5 - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_shinto'].values - wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_shinto'].values)/5)) + wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_shinto'].values)
            total_non_religious_est = float((slope_coef_adj * ((wrp_df.loc[(wrp_df['year'] == year + (5 - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_non_religious'].values - wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_non_religious'].values)/5)) + wrp_df.loc[(wrp_df['year'] == (year - slope_coef_adj)) & (wrp_df['c_code_1'] == c_code)]['total_non_religious'].values)
            row_list.append([year, c_code, c_code_dic[c_code], total_protestant_est, total_catholic_est, total_christian_est, total_jewish_est, total_sunni_est, total_shia_est, total_islam_est, total_buddhist_est, total_hindu_est, total_sikh_est, total_shinto_est, total_non_religious_est])

In [42]:
wrp_df = pd.DataFrame(row_list)

wrp_df.columns = ['year', 'c_code_1', 'state_name_1', 'total_protestant',
         'total_catholic', 'total_christian', 'total_jewish',
         'total_sunni', 'total_shia', 'total_islam',
         'total_buddhist', 'total_hindu', 'total_sikh',
         'total_shinto', 'total_non_religious']

In [43]:
wrp_df.to_pickle('pickle/wrp_df.pkl')

In [44]:
wrp_df.head()

,year,c_code_1,state_name_1,total_protestant,total_catholic,total_christian,total_jewish,total_sunni,total_shia,total_islam,total_buddhist,total_hindu,total_sikh,total_shinto,total_non_religious
0,1945.00,2,USA,66069671.00,38716742.00,110265118.00,4641182.00,0.00,0.00,0.00,1601218.00,0.00,0.00,0.00,22874544.00
1,1946.00,2,United States of America,67473753.40,39500570.00,112810898.20,4931113.00,0.00,0.00,0.00,1450113.50,0.00,0.00,0.00,22813261.20
2,1947.00,2,United States of America,68877835.80,40284398.00,115356678.40,5221044.00,0.00,0.00,0.00,1299009.00,0.00,0.00,0.00,22751978.40
3,1948.00,2,United States of America,70281918.20,41068226.00,117902458.60,5510975.00,0.00,0.00,0.00,1147904.50,0.00,0.00,0.00,22690695.60
4,1949.00,2,United States of America,71686000.60,41852054.00,120448238.80,5800906.00,0.00,0.00,0.00,996800.00,0.00,0.00,0.00,22629412.80


In [45]:
len(wrp_df)

13200

In [46]:
wrp_df[(wrp_df['c_code_1'] == 40) & (wrp_df['year'] == 1985)]

,year,c_code_1,state_name_1,total_protestant,total_catholic,total_christian,total_jewish,total_sunni,total_shia,total_islam,total_buddhist,total_hindu,total_sikh,total_shinto,total_non_religious
238,1985.00,40,CUB,403920.00,7151000.00,7554920.00,1600.00,0.00,0.00,5666.67,6000.00,15000.00,0.00,0.00,2502000.00


In [47]:
one_df = pd.merge(trade_df_group, mat_cap_df, how = 'outer', on = ['c_code_1', 'year'])

one_df.rename({'state_name_1_x': 'state_name_1'}, axis = 1, inplace = True)

one_df.drop(['state_name_1_y'], axis = 1, inplace = True)

In [48]:
one_df.head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score
0,2,United States of America,1870,36.00,256170000.00,359400000.00,13128000.00,50000.00,3384000000.00,75558000000.00,39905000.00,4130000.00,0.10
1,2,United States of America,1871,35.00,319960000.00,430510000.00,11811000.00,42000.00,3470000000.00,79092000000.00,40938000.00,4302000.00,0.10
2,2,United States of America,1872,32.00,313530000.00,353040000.00,14246000.00,42000.00,5180000000.00,96958000000.00,41972000.00,4481000.00,0.12
3,2,United States of America,1873,32.00,392890000.00,542810000.00,15014000.00,43000.00,5204000000.00,107348000000.00,43006000.00,4667000.00,0.12
4,2,United States of America,1874,32.00,317460000.00,553420000.00,12899000.00,44000.00,4878000000.00,103472000000.00,44040000.00,4862000.00,0.11


In [49]:
two_df = pd.merge(one_df, alliance_df_group, how = 'outer', on = ['c_code_1', 'year'])

two_df.rename({'state_name_1_x': 'state_name_1'}, axis = 1, inplace = True)

two_df.drop(['state_name_1_y'], axis = 1, inplace = True)

In [50]:
two_df.head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties
0,2,United States of America,1870,36.00,256170000.00,359400000.00,13128000.00,50000.00,3384000000.00,75558000000.00,39905000.00,4130000.00,0.10,nan,nan,nan,nan,nan,nan,nan
1,2,United States of America,1871,35.00,319960000.00,430510000.00,11811000.00,42000.00,3470000000.00,79092000000.00,40938000.00,4302000.00,0.10,nan,nan,nan,nan,nan,nan,nan
2,2,United States of America,1872,32.00,313530000.00,353040000.00,14246000.00,42000.00,5180000000.00,96958000000.00,41972000.00,4481000.00,0.12,nan,nan,nan,nan,nan,nan,nan
3,2,United States of America,1873,32.00,392890000.00,542810000.00,15014000.00,43000.00,5204000000.00,107348000000.00,43006000.00,4667000.00,0.12,nan,nan,nan,nan,nan,nan,nan
4,2,United States of America,1874,32.00,317460000.00,553420000.00,12899000.00,44000.00,4878000000.00,103472000000.00,44040000.00,4862000.00,0.11,nan,nan,nan,nan,nan,nan,nan


In [51]:
three_df = pd.merge(two_df, mid_df_group, how = 'outer', on = ['c_code_1', 'year'])

three_df.drop(['state_name_abb_a'], axis = 1, inplace = True)

In [52]:
three_df.head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode
0,2,United States of America,1870,36.00,256170000.00,359400000.00,13128000.00,50000.00,3384000000.00,75558000000.00,39905000.00,4130000.00,0.10,nan,nan,nan,nan,nan,nan,nan,2.00,2.00,1.00,0.06,0.00,0.50,0.00,0.00,0,0,4,3,"[8, 19]","[3, 4]"
1,2,United States of America,1871,35.00,319960000.00,430510000.00,11811000.00,42000.00,3470000000.00,79092000000.00,40938000.00,4302000.00,0.10,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2,2,United States of America,1872,32.00,313530000.00,353040000.00,14246000.00,42000.00,5180000000.00,96958000000.00,41972000.00,4481000.00,0.12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
3,2,United States of America,1873,32.00,392890000.00,542810000.00,15014000.00,43000.00,5204000000.00,107348000000.00,43006000.00,4667000.00,0.12,nan,nan,nan,nan,nan,nan,nan,2.00,1.00,1.50,0.07,0.00,1.00,0.00,0.50,"[0, 2]",0,"[3, 5]","[1, 3]","[2, 14]","[2, 3]"
4,2,United States of America,1874,32.00,317460000.00,553420000.00,12899000.00,44000.00,4878000000.00,103472000000.00,44040000.00,4862000.00,0.11,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
four_df = pd.merge(three_df, wrp_df, how = 'outer', on = ['c_code_1', 'year'])

four_df.rename({'state_name_1_x': 'state_name_1'}, axis = 1, inplace = True)

four_df.drop(['state_name_1_y'], axis = 1, inplace = True)

In [54]:
four_df.head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode,total_protestant,total_catholic,total_christian,total_jewish,total_sunni,total_shia,total_islam,total_buddhist,total_hindu,total_sikh,total_shinto,total_non_religious
0,2,United States of America,1870,36.00,256170000.00,359400000.00,13128000.00,50000.00,3384000000.00,75558000000.00,39905000.00,4130000.00,0.10,nan,nan,nan,nan,nan,nan,nan,2.00,2.00,1.00,0.06,0.00,0.50,0.00,0.00,0,0,4,3,"[8, 19]","[3, 4]",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2,United States of America,1871,35.00,319960000.00,430510000.00,11811000.00,42000.00,3470000000.00,79092000000.00,40938000.00,4302000.00,0.10,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2,United States of America,1872,32.00,313530000.00,353040000.00,14246000.00,42000.00,5180000000.00,96958000000.00,41972000.00,4481000.00,0.12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2,United States of America,1873,32.00,392890000.00,542810000.00,15014000.00,43000.00,5204000000.00,107348000000.00,43006000.00,4667000.00,0.12,nan,nan,nan,nan,nan,nan,nan,2.00,1.00,1.50,0.07,0.00,1.00,0.00,0.50,"[0, 2]",0,"[3, 5]","[1, 3]","[2, 14]","[2, 3]",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,2,United States of America,1874,32.00,317460000.00,553420000.00,12899000.00,44000.00,4878000000.00,103472000000.00,44040000.00,4862000.00,0.11,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [55]:
df = pd.merge(four_df, gov_df, how = 'inner', on = ['c_code_1', 'year'])

df = df.sort_values(by = 'year', ascending = True).reset_index()

df.rename({'state_name_1_x': 'state_name_1'}, axis = 1, inplace = True)

df.drop(['state_name_1_y', 'index'], axis = 1, inplace = True)

In [56]:
df.head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode,total_protestant,total_catholic,total_christian,total_jewish,total_sunni,total_shia,total_islam,total_buddhist,total_hindu,total_sikh,total_shinto,total_non_religious,territory_change,territory_change_year,election_type,legislation_type,legislature_status,party_legal_status,party_existance,party_existance_outside_regime,legislature_parties,incumbent_type,collective_leadership,num_leadership_changes,leader_tenure,military_leader,royal_leader,nominal_vs_eff_diff,communist_leader,leader_died,democratic_regime,cabinet_assembly,popular_election,regime_type,transition_democracy,transition_dictatorship,age_govt,num_transitions_ever
0,2,United States of America,1946,65.00,160000000.00,14.40,45133984000.00,3030000.00,120842000000.00,2376288000000.00,141389000.00,39725000.00,0.36,19.00,0.00,0.00,19.00,0.00,0.00,19.00,2.00,1.00,1.50,0.27,0.00,0.50,0.00,0.00,0,"[0, 1]",5,3,"[7, 14]",3,67473753.40,39500570.00,112810898.20,4931113.00,0.00,0.00,0.00,1450113.50,0.00,0.00,0.00,22813261.20,0.00,0.00,1.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,2.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,2.00,0.00,0.00,77.00,0.00
1,325,Italy,1946,65.00,0.00,0.00,213807000.00,260000.00,2306000000.00,18536000000.00,45380000.00,8885000.00,0.02,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,0.00,44308156.80,44501118.40,46390.20,0.00,0.00,2429.40,965.60,0.00,0.00,0.00,930473.80,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00,1.00
2,660,Lebanon,1946,65.00,0.00,0.00,5111000.00,4000.00,0.00,16000000.00,1156000.00,199000.00,0.00,6.00,0.00,4.00,0.00,0.00,6.00,6.00,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,12134.00,424690.00,625385.60,0.00,327681.40,226212.40,553893.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,2.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00
3,140,Brazil,1946,65.00,0.00,0.00,309848000.00,119000.00,686000000.00,11118000000.00,47313000.00,5521000.00,0.01,19.00,0.00,0.00,19.00,0.00,0.00,19.00,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,1480085.80,45049869.60,46546417.80,34978.00,0.00,0.00,0.00,106928.80,0.00,0.00,0.00,150829.80,0.00,0.00,1.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,2.00,1.00,0.00,1.00,0.00
4,339,Albania,1946,65.00,0.00,0.00,11946000.00,60000.00,0.00,402000000.00,1138000.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,2.00,1.00,1.00,0.67,0.00,1.00,0.00,0.00,0,0,"[0, 5]","[0, 3]",19,4,0.00,141442.40,382323.20,571.40,0.00,0.00,755892.00,0.00,0.00,0.00,0.00,1240.20,0.00,0.00,3.00,2.00,2.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,3.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,4.00,0.00,0.00,35.00,0.00


In [57]:
df.loc[df['export_dollars'] == 0, 'export_dollars'] = None
df.loc[df['import_dollars'] == 0, 'import_dollars'] = None
df.loc[df['cinc_score'] == 0, 'cinc_score'] = None

df.loc[df['num_alliances'].isnull(), 'num_alliances'] = 0
df.loc[df['pre_1816_alliances'].isnull(), 'pre_1816_alliances'] = 0
df.loc[df['num_in_effect_1231_2012'].isnull(), 'num_in_effect_1231_2012'] = 0
df.loc[df['defense_treaties'].isnull(), 'defense_treaties'] = 0
df.loc[df['neutrality_treaties'].isnull(), 'neutrality_treaties'] = 0
df.loc[df['nonaggression_treaties'].isnull(), 'nonaggression_treaties'] = 0
df.loc[df['entente_treaties'].isnull(), 'entente_treaties'] = 0
df.loc[df['num_conflicts'].isnull(), 'num_conflicts'] = 0
df.loc[df['avg_cum_duration'].isnull(), 'avg_cum_duration'] = 0
df.loc[df['num_wars'].isnull(), 'num_wars'] = 0
df.loc[df['state_fatality_bin_mode'].isnull(), 'state_fatality_bin_mode'] = 0
df.loc[df['revision_type_mode'].isnull(), 'revision_type_mode'] = 0

df.loc[df['highest_mca_mode'].isnull(), 'highest_mca_mode'] = 1
df.loc[df['highest_hostility_mode'].isnull(), 'highest_hostility_mode'] = 1

# df.loc[df['avg_with'].isnull(), 'avg_with'] = 0
# df.loc[df['avg_against'].isnull(), 'avg_against'] = 0
# df.loc[df['origin_participant_pct'].isnull(), 'origin_participant_pct'] = 0
# df.loc[df['revision_pct'].isnull(), 'revision_pct'] = 0

In [58]:
df.to_pickle('pickle/df.pkl')

In [59]:
df[df['state_name_1'].isnull()].head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode,total_protestant,total_catholic,total_christian,total_jewish,total_sunni,total_shia,total_islam,total_buddhist,total_hindu,total_sikh,total_shinto,total_non_religious,territory_change,territory_change_year,election_type,legislation_type,legislature_status,party_legal_status,party_existance,party_existance_outside_regime,legislature_parties,incumbent_type,collective_leadership,num_leadership_changes,leader_tenure,military_leader,royal_leader,nominal_vs_eff_diff,communist_leader,leader_died,democratic_regime,cabinet_assembly,popular_election,regime_type,transition_democracy,transition_dictatorship,age_govt,num_transitions_ever
22,223,NaN,1946,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,77.00,0.00
26,305,NaN,1946,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,1.00,2.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,1.00,0.00,0.00,2.00,1.00
27,740,NaN,1946,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,122132.80,109484.80,693413.60,0.00,0.00,0.00,0.00,51235564.00,4405.60,0.00,57273587.80,818164.00,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,4.00,1.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
28,760,NaN,1946,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,5.00,0.00,0.00,40.00,0.00
115,223,NaN,1947,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,3.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,78.00,0.00


In [60]:
len(df['c_code_1'].unique())

198

In [61]:
len(df)

9147

In [62]:
df.columns

Index(['c_code_1', 'state_name_1', 'year', 'num_trade_states',
       'export_dollars', 'import_dollars', 'military_expenditure',
       'military_personnel', 'iron_steel_prod', 'prim_energy_consumption',
       'total_pop', 'urban_pop', 'cinc_score', 'num_alliances',
       'pre_1816_alliances', 'num_in_effect_1231_2012', 'defense_treaties',
       'neutrality_treaties', 'nonaggression_treaties', 'entente_treaties',
       'num_conflicts', 'avg_with', 'avg_against', 'avg_cum_duration',
       'num_wars', 'origin_participant_pct', 'ongoing_2010', 'revision_pct',
       'revision_type_mode', 'state_fatality_bin_mode', 'outcome_mode',
       'settlement_type_mode', 'highest_mca_mode', 'highest_hostility_mode',
       'total_protestant', 'total_catholic', 'total_christian', 'total_jewish',
       'total_sunni', 'total_shia', 'total_islam', 'total_buddhist',
       'total_hindu', 'total_sikh', 'total_shinto', 'total_non_religious',
       'territory_change', 'territory_change_year', 'elect

In [63]:
df[([s % 5 != 0 for s in df['year']])].head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode,total_protestant,total_catholic,total_christian,total_jewish,total_sunni,total_shia,total_islam,total_buddhist,total_hindu,total_sikh,total_shinto,total_non_religious,territory_change,territory_change_year,election_type,legislation_type,legislature_status,party_legal_status,party_existance,party_existance_outside_regime,legislature_parties,incumbent_type,collective_leadership,num_leadership_changes,leader_tenure,military_leader,royal_leader,nominal_vs_eff_diff,communist_leader,leader_died,democratic_regime,cabinet_assembly,popular_election,regime_type,transition_democracy,transition_dictatorship,age_govt,num_transitions_ever
0,2,United States of America,1946,65.00,160000000.00,14.40,45133984000.00,3030000.00,120842000000.00,2376288000000.00,141389000.00,39725000.00,0.36,19.00,0.00,0.00,19.00,0.00,0.00,19.00,2.00,1.00,1.50,0.27,0.00,0.50,0.00,0.00,0,"[0, 1]",5,3,"[7, 14]",3,67473753.40,39500570.00,112810898.20,4931113.00,0.00,0.00,0.00,1450113.50,0.00,0.00,0.00,22813261.20,0.00,0.00,1.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,2.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,2.00,0.00,0.00,77.00,0.00
1,325,Italy,1946,65.00,nan,nan,213807000.00,260000.00,2306000000.00,18536000000.00,45380000.00,8885000.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,44308156.80,44501118.40,46390.20,0.00,0.00,2429.40,965.60,0.00,0.00,0.00,930473.80,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00,1.00
2,660,Lebanon,1946,65.00,nan,nan,5111000.00,4000.00,0.00,16000000.00,1156000.00,199000.00,0.00,6.00,0.00,4.00,0.00,0.00,6.00,6.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,12134.00,424690.00,625385.60,0.00,327681.40,226212.40,553893.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,2.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00
3,140,Brazil,1946,65.00,nan,nan,309848000.00,119000.00,686000000.00,11118000000.00,47313000.00,5521000.00,0.01,19.00,0.00,0.00,19.00,0.00,0.00,19.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,1480085.80,45049869.60,46546417.80,34978.00,0.00,0.00,0.00,106928.80,0.00,0.00,0.00,150829.80,0.00,0.00,1.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,2.00,1.00,0.00,1.00,0.00
4,339,Albania,1946,65.00,nan,nan,11946000.00,60000.00,0.00,402000000.00,1138000.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,2.00,1.00,1.00,0.67,0.00,1.00,0.00,0.00,0,0,"[0, 5]","[0, 3]",19,4,0.00,141442.40,382323.20,571.40,0.00,0.00,755892.00,0.00,0.00,0.00,0.00,1240.20,0.00,0.00,3.00,2.00,2.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,3.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,4.00,0.00,0.00,35.00,0.00


In [64]:
df[df['military_expenditure'].isnull()].head()

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode,total_protestant,total_catholic,total_christian,total_jewish,total_sunni,total_shia,total_islam,total_buddhist,total_hindu,total_sikh,total_shinto,total_non_religious,territory_change,territory_change_year,election_type,legislation_type,legislature_status,party_legal_status,party_existance,party_existance_outside_regime,legislature_parties,incumbent_type,collective_leadership,num_leadership_changes,leader_tenure,military_leader,royal_leader,nominal_vs_eff_diff,communist_leader,leader_died,democratic_regime,cabinet_assembly,popular_election,regime_type,transition_democracy,transition_dictatorship,age_govt,num_transitions_ever
22,223,NaN,1946,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,77.00,0.00
24,40,Cuba,1946,65.00,nan,nan,nan,18000.00,0.00,162000000.00,5040000.00,1224000.00,0.00,19.00,0.00,0.00,19.00,0.00,0.00,19.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,192862.40,4502960.00,4695822.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,14.00,1.00,0.00,1.00,0.00,1.00,1.00,0.00,1.00,2.00,0.00,0.00,11.00,0.00
26,305,NaN,1946,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,1.00,2.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,1.00,0.00,0.00,2.00,1.00
27,740,NaN,1946,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,122132.80,109484.80,693413.60,0.00,0.00,0.00,0.00,51235564.00,4405.60,0.00,57273587.80,818164.00,0.00,0.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,4.00,1.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
28,760,NaN,1946,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,5.00,0.00,0.00,40.00,0.00


In [65]:
df[(df['year'] == 1952) & (df['c_code_1'] == 40)]

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode,total_protestant,total_catholic,total_christian,total_jewish,total_sunni,total_shia,total_islam,total_buddhist,total_hindu,total_sikh,total_shinto,total_non_religious,territory_change,territory_change_year,election_type,legislation_type,legislature_status,party_legal_status,party_existance,party_existance_outside_regime,legislature_parties,incumbent_type,collective_leadership,num_leadership_changes,leader_tenure,military_leader,royal_leader,nominal_vs_eff_diff,communist_leader,leader_died,democratic_regime,cabinet_assembly,popular_election,regime_type,transition_democracy,transition_dictatorship,age_govt,num_transitions_ever
469,40,Cuba,1952,76.00,643930000.20,758819999.80,56000000.00,20000.00,0.00,1286000000.00,5759000.00,1416000.00,0.00,20.00,0.00,0.00,20.00,0.00,20.00,20.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,229417.60,5163240.00,5392657.60,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,0.00,0.00,2.00,2.00,2.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,4.00,0.00,1.00,1.00,1.00


In [ ]:
# impute for total non-religious
# do any other religion values not make sense as 0?
# use threshold for first non-zero
# maybe a relevant 0 is any 0 followed by a number greater than 10,000?

In [67]:
df[df['c_code_1'] == 2]

,c_code_1,state_name_1,year,num_trade_states,export_dollars,import_dollars,military_expenditure,military_personnel,iron_steel_prod,prim_energy_consumption,total_pop,urban_pop,cinc_score,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties,num_conflicts,avg_with,avg_against,avg_cum_duration,num_wars,origin_participant_pct,ongoing_2010,revision_pct,revision_type_mode,state_fatality_bin_mode,outcome_mode,settlement_type_mode,highest_mca_mode,highest_hostility_mode,total_protestant,total_catholic,total_christian,total_jewish,total_sunni,total_shia,total_islam,total_buddhist,total_hindu,total_sikh,total_shinto,total_non_religious,territory_change,territory_change_year,election_type,legislation_type,legislature_status,party_legal_status,party_existance,party_existance_outside_regime,legislature_parties,incumbent_type,collective_leadership,num_leadership_changes,leader_tenure,military_leader,royal_leader,nominal_vs_eff_diff,communist_leader,leader_died,democratic_regime,cabinet_assembly,popular_election,regime_type,transition_democracy,transition_dictatorship,age_govt,num_transitions_ever
0,2,United States of America,1946,65.00,160000000.00,14.40,45133984000.00,3030000.00,120842000000.00,2376288000000.00,141389000.00,39725000.00,0.36,19.00,0.00,0.00,19.00,0.00,0.00,19.00,2.00,1.00,1.50,0.27,0.00,0.50,0.00,0.00,0,"[0, 1]",5,3,"[7, 14]",3,67473753.40,39500570.00,112810898.20,4931113.00,0.00,0.00,0.00,1450113.50,0.00,0.00,0.00,22813261.20,0.00,0.00,1.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,2.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,2.00,0.00,0.00,77.00,0.00
135,2,United States of America,1947,67.00,76000000.00,9.54,14315999000.00,1583000.00,154030000000.00,2631554000000.00,144126000.00,40857000.00,0.31,39.00,0.00,19.00,39.00,0.00,20.00,39.00,0.00,nan,nan,0.00,0.00,nan,nan,nan,0,0,NaN,NaN,1,1,68877835.80,40284398.00,115356678.40,5221044.00,0.00,0.00,0.00,1299009.00,0.00,0.00,0.00,22751978.40,0.00,0.00,1.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,3.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,2.00,0.00,0.00,78.00,0.00
147,2,United States of America,1948,71.00,6926000000.00,11343750000.00,10960998000.00,1446000.00,160826000000.00,2792788000000.00,147208000.00,42019000.00,0.29,20.00,0.00,19.00,20.00,0.00,20.00,20.00,1.00,1.00,3.00,0.78,0.00,1.00,0.00,0.00,0,0,0,0,15,4,70281918.20,41068226.00,117902458.60,5510975.00,0.00,0.00,0.00,1147904.50,0.00,0.00,0.00,22690695.60,0.00,0.00,1.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,4.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,2.00,0.00,0.00,79.00,0.00
292,2,United States of America,1949,74.00,6483000000.00,10386040000.00,13503000000.00,1615000.00,141480000000.00,2398688000000.00,149767000.00,43216000.00,0.27,31.00,0.00,30.00,31.00,0.00,31.00,31.00,3.00,1.00,1.67,0.62,0.00,1.00,0.00,0.00,0,0,0,0,1,1,71686000.60,41852054.00,120448238.80,5800906.00,0.00,0.00,0.00,996800.00,0.00,0.00,0.00,22629412.80,0.00,0.00,1.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,5.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,2.00,0.00,0.00,80.00,0.00
312,2,United States of America,1950,74.00,8228999976.00,8913779980.00,14559000000.00,1460000.00,175696000000.00,2483672000000.00,152271000.00,43964000.00,0.28,31.00,0.00,30.00,31.00,0.00,31.00,31.00,5.00,1.60,7.40,0.32,2.00,0.40,0.00,0.20,0,0,"[0, 7]","[0, 3]","[1, 22]","[1, 5]",73090083.00,42635882.00,122994019.00,6090837.00,0.00,0.00,0.00,845695.50,0.00,0.00,0.00,22568130.00,0.00,0.00,1.00,2.00,2.00,2.00,2.00,2.00,2.00,0.00,0.00,0.00,6.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,2.00,0.00,0.00,81.00,0.00
419,2,United States of America,1951,75.00,10192799970.00,12816640020.00,33398000000.00,3249000.00,190870000000.00,2651896000000.00,154878000.00,44711000.00,0.32,37.00,0.00,35.00,37.00,0.00,35.00,35.00,7.00,3.00,6.29,0.87,2.00,0.14,0.00,0.14,0,0,5,1,7,3,74330992.00,43389179.20,125195569.20,5939336.00,0.00,0.00,0.00,694591.00,0.00,0.00,0.00,22715212.00,0.00,0.00,1.00,2.00,2.00,2.00,2.